# Exp1: Observation noise (thin notebook)

Purpose:
- Apply Poisson/NegBin observation models to I(t) and fit classical MLE on a small subset.
- Mirrors the logic in `scripts/exp1_noise.py` but kept small for quick checks.

How to use:
1) Set noise type and parameters (rho, k) in the config cell.
2) Run all cells to evaluate a small subset.
3) For full runs and CSV outputs, use:
   `python scripts/exp1_noise.py --noise poisson --rho 0.7 --train-mode clean`


In [ ]:
from pathlib import Path
import sys
import numpy as np

repo_root = Path.cwd()
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

from src.sir.config import DEFAULTS, set_global_seed
from src.sir.datasets import load_sir_pkl, build_Xy_I_only, train_val_test_split
from src.sir.noise import observe_poisson, observe_negbin
from src.sir.baseline import fit_poisson_mle, fit_negbin_mle
from src.sir.metrics import per_param_metrics, timing_summary


In [ ]:
# Configuration
seed = 42
set_global_seed(seed)
rng = np.random.default_rng(seed)

data_path = DEFAULTS.data_path
limit = 5000
test_size = 0.10
val_size = 0.10
n_starts = 3
max_test = 200

noise = 'poisson'  # or 'negbin'
rho = 0.7
k = 20.0


In [ ]:
data = load_sir_pkl(data_path, limit=limit, rng=rng)
X, y = build_Xy_I_only(data, normalize=None)

splits = train_val_test_split(
    X, y, test_size=test_size, val_size=val_size, rng=rng, return_indices=True
)
X_test = splits['X_test']
y_test = splits['y_test']

if noise == 'poisson':
    X_obs = observe_poisson(X_test, rho=rho, rng=rng)
else:
    X_obs = observe_negbin(X_test, rho=rho, k=k, rng=rng)


In [ ]:
idx = rng.choice(X_obs.shape[0], size=min(max_test, X_obs.shape[0]), replace=False)
X_fit = X_obs[idx]
y_fit = y_test[idx]

preds = []
times = []
for i in range(X_fit.shape[0]):
    if noise == 'poisson':
        fit = fit_poisson_mle(X_fit[i], rho=rho, n_starts=n_starts, rng=np.random.default_rng(seed + i))
    else:
        fit = fit_negbin_mle(X_fit[i], rho=rho, k=k, n_starts=n_starts, rng=np.random.default_rng(seed + i))
    preds.append(fit.params[:2])
    times.append(sum(fit.times))

preds = np.asarray(preds)
metrics = per_param_metrics(y_fit, preds)
metrics.update(timing_summary(np.asarray(times)))
metrics


Run full benchmark with:

```bash
python scripts/exp1_noise.py --noise poisson --rho 0.7 --train-mode clean
```
